In [22]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [34]:
epoch_ct = 50
batch_size = 86

In [24]:
MNIST_TRAIN_DIR = os.getcwd() + '/mnist_train.csv'
MNIST_TEST_DIR = os.getcwd() + '/mnist_test.csv'

def ImportMNISTDataFromDirectory(train_dir, test_dir):
    mnist_test = pd.read_csv(test_dir)
    mnist_train = pd.read_csv(train_dir)

    mnist = pd.concat([mnist_train, mnist_test])
    mnist.columns = mnist_train.columns

    X_df = mnist.drop(['label'], axis=1)
    y_df = mnist.label
    return X_df, y_df

In [25]:
X_df, y_df = ImportMNISTDataFromDirectory(MNIST_TRAIN_DIR, MNIST_TEST_DIR)

X_df = X_df.values.reshape(-1, 28, 28, 1)
y_df = tf.keras.utils.to_categorical(y_df, num_classes=10)


In [26]:
# Divide into validation and training data
X_t, X_v, y_t, y_v = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [56]:
rescale_data = keras.Sequential([keras.layers.Rescaling(1/255.)])

train_ds = tf.data.Dataset.from_tensor_slices((X_t, y_t))
vald_ds = tf.data.Dataset.from_tensor_slices((X_v, y_v))

train_ds = train_ds.map(lambda x, y: (rescale_data(x), y))
train_ds = train_ds.shuffle(1000)
train_ds.batch(batch_size)

vald_ds = vald_ds.map(lambda x, y: (rescale_data(x), y))
vald_ds = vald_ds.shuffle(1000)
vald_ds.batch(batch_size)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [59]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    # keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2, padding='same'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    # keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),    
    keras.layers.MaxPool2D(pool_size=2, padding='same'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    # keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2, padding='same'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.output_shape

(None, 10)

In [51]:
flayer = keras.layers.Flatten()
dlayer = keras.layers.Dense(256, activation='relu')
x = tf.constant(np.random.random((86, 4, 4, 128)), dtype=tf.float32)
xf = flayer(x)
xd = dlayer(xf)
xd.shape

TensorShape([86, 256])

In [62]:
lr_reduction = keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=2,
    min_lr=0.0000001
)

hist = model.fit(
    train_ds,
    validation_data = vald_ds,
    epochs = epoch_ct,
    callbacks=[lr_reduction],
    shuffle=True,
)

Epoch 1/50


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_t

In [ ]:
future = model.evaluate(vald_ds, verbose=2)

313/313 - 1s - loss: 0.1246 - accuracy: 0.9730 - 735ms/epoch - 2ms/step


In [ ]:
pd.DataFrame(hist.history).plot()